# Big Data Strategies

## Step 1: Diagnosing if you have "Big Data"

Oddly, this is not actually a straight-forward question for two reasons:

1.  **Programs often copy data in order to manipulate it.** When you, say, sort an array, what your computer actually does is create a new empty array, then move over entries from your original array into the new array one entry at a time (in sorted order). While it does then erase the original array, while it's working it effectively has two copies of your array. As a result, the fact that you have 16gb of RAM doesn't mean you can easily work with a 14gb file. As a general rule, you need *at least* two times as much memory as your file takes up when first loaded. 
2. **The size of data changes as you change its format.** For example, if you're reading data from an inefficient file format (like a .csv text file), it may be the case that a file that is 10gb on disk may actually be only a few gb once you load it into a program like Python that knows how to store data more efficiently. 

### So how do I check to see if my data is fitting in memory?

If your program starts using more space than you have main memory, your operating system will usually just start using your harddrive for extra space without telling you (this is called "virtual memory", and is nice in that it prevents your computer from crashing, though it will slow stuff down a lot). As a result, you won't always get an error message if you try and load a file to is much bigger than main memory. 

To avoid this, use one of the tools provided by your operating system to monitor whether it's using the harddrive:

- **OSX:** OSX comes with a program called [Activity Monitor](https://support.apple.com/en-us/HT201464#memory) in the Applications > Utilities folder. The memory tab of Activity Monitor has a graph at the bottom called "Memory Pressure". If this is green, you're good. If it's yellow or red, you're actively going back and forth to your harddrive. (Note your computer may be using "virtual memory" without affecting performance if it's just storing away things you aren't actively using).
- **Windows:** Windows has a program called ["Resource Monitor"](http://en.wikipedia.org/wiki/Resource_Monitor) that shows data on memory use. If you get near 100% Used Physical Memory, your computer is probably using the harddrive.

## Step 2: Pick a Strategy

If you have big data, you basically have four options:

1. Use chunking to trim and thin out your data so it *does* fit in memory (this works if the data you were given is huge, but the final analysis dataset you want to work with is small). 
2. Buy more memory. Seriously, consider it. 
3. Minimize the penalties of working off your harddrive (not usually practical for data science)
4. Break your job into pieces and distribute over multiple machines. 


### Strategy 1: Trim till it fits

At least in my experience, *most* of the time people have a dataset that's too big to load into main memory, they're in a situation where the *raw* data they want to work with is too big for main memory, but the dataset they are trying to create for analysis will not be. This can happen when the raw data you're working with:

- has lots of variables you don't need for your analysis, 
- has lots of observations you don't need for your analysis,
- the data is at a more granular level then you need for analysis (e.g. it's individual level, but you only need averages for different groups), or
- where data is formatted in a really efficient manner where better data management would allow it to fit. 

In these situations, the go-to strategy is **chunking**, which is the practice of:

- loading only a small part of your data, preprocessing it (dropping extra variables, dropping extra observations, taking averages, and/or storing the data in more efficient formats), and saving it away,
- then recombining all the now substantially smaller bits into a new analysis dataset that has everything you want and fits into data. 

Chunking is pretty easy with most data formats. In pandas, for example, you can specify what rows you want to import using either the `nrows` (to specify you only want the first `n` rows read into memory), or you can use the combination of `skiprows` and `nrows` to get a specific set of rows (i.e. "start right after the number of rows in `skiprows`, then take `nrows` from there). Or if you really want to be fancy, you can actually just use `read_csv` as an interator using `iterator=True` and `chunksize`, which will return an object that you can loop over and which pulls in the number of rows you specify in `chunksize` each pass of the loop. 

While you usually want to start by just looking at the first subset of rows in a dataset (so you can see the whole data structure), read functions can also make chunking unnecessary if you just want to do some standard thinning. For example, `read_csv` has a `usecols` option where you can specify what subset of columns you want to keep (this isn't unique to csvs -- `read_hdf` has similar options), and a `dtypes` option for specifying the data format in which you want data stored *as it's streaming in* from disk (if you want to use more efficient data formats than the defaults -- e.g. using `Float32` instead of `Float64` when you know you don't need all the precision offered by `Float64`. See [numbers in machines](ints_and_floats.ipynb) for a refresher on tradeoffs from different data types). 

### Think you can do while chunking to reduce memory use



## Avoid Spillover
Working with data that doesn't fit into RAM is a ''headache''. If there's anyway you can avoid it, you should, which can potentially be accomplished via two methods:

- '''Drop variables or store them more efficiently:''' If you're used to small datasets, you may be in the habit of always carrying around all the variables in a survey or dataset. But if you only keep the variables you absolutely need, or find ways to store them more efficiently, you can often stay within your size limits. For example, String variables take up lots of space -- if you can, turn them into numeric variables (see [[Data Types]] for more on data types).
- '''Get more RAM''': RAM isn't cheap, but nor is your time, and buying more RAM can save you an amazing amount of trouble. And if you can't afford to buy your own, you may find that it's cheaper to rent a computer with more RAM from cloud services like Amazon, which are often surprisingly affordable! For more, see the page on [[Cloud Computing Resources]].

## Chunking
One way to manipulate large datasets is by chunking -- loading your data into RAM one chunk (say, 1 million rows) at a time, manipulating it, and then appending it to a new data file. This is relatively straightforward for operations that act only on single rows -- like recoding variables -- but a little harder when you start doing things that involve multiple rows at once (like sorting, or taking averages over groups).


## Big Data Tools


### Easy to work with, but somewhat limited: `dask`

`Dask` is a new tool written for working with data that doesn't fit into memory (and parallelizing operations) for Python. It was written to basically work just like `pandas`, so it's quite easy to get started using. The only catch is that it only supports a certain number of functions at this point, so it will do a lot, but not everything.

* `Dask Overview <https://www.youtube.com/watch?v=1kkFZ4P-XHg>`_
* `Long Tutorial <https://www.youtube.com/watch?v=ieW3G7ZzRZ0>`_

Dask is probably best for *big data*, but I'm not sure it can really handle *very big data* (data you can't put on one computer's harddrive). The authors aim to fix that, but I'm not sure the system is there yet. 

### More involved, but more powerful: `pyspark`


`Spark` is like the second generation of a platform called Hadoop for working with data across lots and lots of software. `pyspark` is a great tool for manipulating `spark` using python. `spark` is *very* powerful, but it's not a tool created for Python users, it's an entire computing system you'll have to learn about to use. 

Though there are attempts to make `PySpark` easier for `pandas` users to use, like `Sparkling Pandas` (`tutorial here <https://www.youtube.com/watch?v=borv_KMI9Ac>`_)

Unlike `Dask`, `Spark` and `PySpark` were built not just for *big data* (data that doesn't fit in RAM), but specifically for *very big data* (data that won't even fit on a single computer's hard drive). So if you have *very big data*, this is probably the way to go. 